# Chapter06 : 학습 관련 기술들

In [ ]:
import numpy as np
import torch

## 1. 매개변수 갱신

### SGD : Stochastic Gradient Descent

In [ ]:
# SGD : Stochastic Gradient Descent
class SGD:
    def __init__(self, lr=0.01):                    # lr : learning rate를 의미
        self.lr = lr
    
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]     # 각 변수의 주소로 접근해 값을 변경

##### Greedy algorithm : 매 선택에서 최적이라 판단되는 것을 선택하는 알고리즘. 최적 해를 구하는 근사적 방법이지만 단점이 많다.<br>SGD는 매 순간 최대의 효율을 얻는 길을 선택한다.

### Momentum : 운동량
> 물체가 경사면을 따라 운동할때에는 가속도(현재 SGD의 미분값)뿐만 아니라 물체의 속도 또한 존재한다. 이를 구현한 방식이 모멘텀이다.

In [ ]:
class Momentum:
    def __init__(self, lr=0.01, momentum = 0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
    
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
        
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]  

### AdaGrad : 모든 변수에 통일된 학습률을 개별 매개변수에 서로 다른 학습률을 적용하는 방식, 학습률 변화
> 기울기가 컸던 파라미터의 학습률은 낮게, 기울기가 작았던 파라미터의 학습률은 크게

In [ ]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
    
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
        
        for key in params.keys():
            self.h[key] += grads[key] ** 2          # 과거 기울기값 ~ self.h ~ 1/학습량
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-07)



### Adam : Adaptive Moment Estimation
>RMSprop와 Momentum을 기반으로 만들어진 optimizer

In [ ]:
# pytorch 에서 사용하는 방법
optimizer = torch.optim.Adam()

## 2. 가중치의 초깃값
> 가중치의 초기값이 모두 0이라면 모든 값들이 똑같이 학습될 것이다. 그렇다면 어떤 값으로 어떻게 초기값을 설정해야하는가?

#### 평균 0을 기준으로 분산을 어떻게 선택하느냐의 문제이다.여기서 분산을 두 가지 방법으로 선택한다.

### Xavier 초기화, Kaiming(He)초기화
> 각각 S자 곡선, ReLU함수를 활성화 함수로 사용할 때 사용되는 초기화 방식이다.

##### 앞 계층의 노드가 n개라 하자. (입력의 경우 n은 배치 사이즈이다.) <br>이 때 Xavier 초기화는 분산을 $1\over\sqrt{n}$<br>He 초기화는 분산을 $\sqrt{2\over{n}}$으로 사용한다.

In [1]:
#torch.nn.init.xavier_normal_()
#torch.nn.init.kaiming_normal_()

## 3. 배치 정규화
> 앞서 보았던 활성화 값의 분포가 되도록 강제하는 방법

##### 순방향에 들어온 입력 데이터(x)의 분포가 평균이 0, 분산이 1이 되도록 정규화한다. 이를 통해 학습을 안정화시킨다.

##### 장점
- 학습 속도
- 초기값 의존도 감소
- 오버피팅 억제(드롭아웃의 필요성 감소)

#### $미니배치B = \set{x_1, x_2, ..., x_m}$<br><br>$\hat{x_i} = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 - \epsilon}}$<br>$y_i = \gamma \hat{x_i} + \beta$
> 미니배치 단위로 정규화 후 스케일($\gamma$)과 시프트($\beta$)를 적용하는 레이어

## 4. 바른 학습을 위해 - 오버피팅
> 오버피팅이란 신경망이 훈련 데이터에 지나치게 적응하여 새로운 데이터에 제대로 대응하지 못하는 상태를 말한다.

요인
- 매개변수가 많고 표현력이 높은 모델
- 훈련 데이터가 적음

### 가중치 감소 : Weight Decay
> 손실함수에 가중치의 크기에 비례한 손실항을 추가한다. 이로서 큰 가중치를 억제한다.

##### $L_{new}(\theta) = L(\theta) + \frac{1}{2}\lambda W^2$
> 여기서 $\lambda$는 하이퍼파라미터

### 드롭아웃 : Dropout
> 훈련 때 무작위 노드를 제거해 계속해서 학습이 잘 되는 노드의 학습을 방해한다. 또 여러 형태의 신경망을 학습시키는 것과 비슷한 효과를 가져온다.

In [23]:
import numpy as np
dropout_ratio = 0.4
mask = np.random.rand(1) > dropout_ratio    # True == 1, False == 0
print(mask, int(mask))
# return y * mask

[ True] 1


/var/folders/2v/b65115v14cvgvjztf82wr9x80000gn/T/ipykernel_71815/2897421905.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(mask, int(mask))


## 5. 적절한 하이퍼파라미터 값 찾기
> 이는 가중치(파라미터)처럼 역전파 알고리즘에 의해 자동으로 업데이트되는 값이 아니다. 그리고 하이퍼파라미터의 가능한 값의 범위는 크기 때문에 추론이 필요하다.

### 데이터셋 분할
- 훈련 데이터 64% : 매개변수 학습
- 검증 데이터 16% : 하이퍼파라미터 성능 평가
- 시험 데이터 20% : 신경망의 범용 성능 평가

- 0단계  하이퍼파라미터 값의 범위를 설정한다.
- 1단계  설정된 범위에서 하이퍼파라미터의 값을 무작위로 추출한다.
- 2단계    1단계에서 샘플링한 하이퍼파라미터 값을 사용하여 학습하고, 검증 데이터로 정확도를 평가한다 (단, 에폭은 작게 설정한다 ).
- 3단계    1단계와 2단계를 특정 횟수 (100회 등 ) 반복하며, 그 정확도의 결과를 보고 하이퍼파라미터의 범위를 좁 힌다.
- 이상을 반복

위의 방법 말고도 **베이즈 최적화**라는 확률을 사용한 최적화 기법이 존재한다.